In [8]:
import pandas as pd
import numpy as np

# Load the dataset
file_path = 'epu_data_final.xlsx'  # Adjust the path if the file is in a different directory
data = pd.read_excel(file_path, parse_dates=['quarter'])

# Set the 'quarter' column as the index if you want to analyze time series
data.set_index('quarter', inplace=True)

# Get descriptive statistics for the numerical variables
descriptive_stats = data.describe()

# If you want to include the variability measure beyond the standard deviation, you can compute the coefficient of variation
coeff_variation = data.std() / data.mean()

# Display the descriptive statistics
print("Descriptive Statistics:\n", descriptive_stats)

# Display the coefficient of variation for each variable
print("\nCoefficient of Variation:\n", coeff_variation)


Descriptive Statistics:
              Close         Open          Low         High        Volume  \
count   480.000000   480.000000   480.000000   480.000000  4.630000e+02   
mean    556.275642   553.411975   517.335789   589.538722  8.509597e+10   
std     773.776670   773.271401   728.530563   814.089834  2.098400e+11   
min       0.077998     0.078347     0.070922     0.087807  5.304100e+04   
25%      25.727500    25.909412    24.345000    27.585000  1.238898e+07   
50%     196.809773   171.917837   164.745336   204.518276  1.184637e+10   
75%     582.326348   578.955000   539.711077   634.272500  6.765651e+10   
max    2667.390000  2609.320000  2553.190000  2755.540000  1.510692e+12   

                gdp         epu  liquidity_1   liquidity_2  
count  4.800000e+02  480.000000   480.000000  4.800000e+02  
mean   1.754895e+06    0.153304     0.144807  8.208215e+10  
std    1.553112e+06    0.163360     0.101824  2.066833e+11  
min    5.351384e+04    0.000000     0.023713  0.000000e

C:\Users\Tam\AppData\Local\Temp/ipykernel_26096/414282170.py:15: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  coeff_variation = data.std() / data.mean()


In [9]:
descriptive_stats.to_excel('descriptive.xlsx')

In [10]:
coeff_variation

Close          1.390995
Open           1.397280
Low            1.408235
High           1.380893
Volume         2.465922
gdp            0.885017
epu            1.065595
liquidity_1    0.703168
liquidity_2    2.518005
dtype: float64

In [11]:
# Compute the coefficient of correlation
correlation_matrix = data.corr()
correlation_matrix.to_excel('correlation.xlsx')

In [12]:
import pandas as pd
from statsmodels.tsa.stattools import adfuller

# Assuming you've loaded your dataset into a DataFrame named `data`
# Example: data = pd.read_excel('epu_data_final.xlsx')

# Function to perform ADF test
def adf_test(series, name='Variable'):
    result = adfuller(series.dropna(), autolag='AIC')  # Drop NA values and select the lag automatically based on AIC
    print(f'ADF Statistic for {name}: {result[0]}')
    print(f'p-value for {name}: {result[1]}')
    print('Critical Values:')
    for key, value in result[4].items():
        print(f'   {key}: {value}')
    print("\n")

# Perform ADF test on the variables
adf_test(data['gdp'], 'GDP')
adf_test(data['epu'], 'EPU')
adf_test(data['liquidity_1'], 'Liquidity 1')
adf_test(data['liquidity_2'], 'Liquidity 2')

ADF Statistic for GDP: -2.1740470975864845
p-value for GDP: 0.21579931965367355
Critical Values:
   1%: -3.4444914328761977
   5%: -2.8677756786103683
   10%: -2.570091378194011


ADF Statistic for EPU: -7.823048552749505
p-value for EPU: 6.5916778735610265e-12
Critical Values:
   1%: -3.444133731419755
   5%: -2.8676183066831036
   10%: -2.5700075029117166


ADF Statistic for Liquidity 1: -5.145301650149465
p-value for Liquidity 1: 1.1358102202001438e-05
Critical Values:
   1%: -3.444250937448703
   5%: -2.867669873870454
   10%: -2.5700349866579657


ADF Statistic for Liquidity 2: -3.028065723161877
p-value for Liquidity 2: 0.032355248350369036
Critical Values:
   1%: -3.4446148284445153
   5%: -2.8678299626609314
   10%: -2.5701203107928157




In [41]:
def perform_granger_full_output(data):
    """
    Perform Granger causality tests for specified variable pairs and print full results.
    The data parameter should be a pandas DataFrame containing the relevant series.
    """
    # List of variable pairs to test
    pairs_to_test = [('gdp', 'epu'), ('gdp', 'liquidity_1'), ('gdp', 'liquidity_2'),
                     ('epu', 'liquidity_1'), ('epu', 'liquidity_2'), ('epu', 'gdp'),
                    ('liquidity_1', 'gdp'), ('liquidity_2', 'gdp'), ('liquidity_1', 'epu'), ('liquidity_2', 'epu')]

    for x, y in pairs_to_test:
#         print(f"\nGranger Causality Test: {x} causing {y}")
        # Perform the Granger causality test
        test_result = grangercausalitytests(data[[x, y]], maxlag=5, verbose=True)

# To use this function, make sure your 'data' DataFrame is properly defined with the necessary columns,
# and then simply call the function:
# perform_granger_full_output(data)


In [43]:
perform_granger_full_output(data)


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0003  , p=0.9856  , df_denom=476, df_num=1
ssr based chi2 test:   chi2=0.0003  , p=0.9855  , df=1
likelihood ratio test: chi2=0.0003  , p=0.9855  , df=1
parameter F test:         F=0.0003  , p=0.9856  , df_denom=476, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.0991  , p=0.9056  , df_denom=473, df_num=2
ssr based chi2 test:   chi2=0.2004  , p=0.9047  , df=2
likelihood ratio test: chi2=0.2003  , p=0.9047  , df=2
parameter F test:         F=0.0991  , p=0.9056  , df_denom=473, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=0.0472  , p=0.9864  , df_denom=470, df_num=3
ssr based chi2 test:   chi2=0.1437  , p=0.9861  , df=3
likelihood ratio test: chi2=0.1437  , p=0.9861  , df=3
parameter F test:         F=0.0472  , p=0.9864  , df_denom=470, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=0.1203  , p=0.9752  

parameter F test:         F=1.1333  , p=0.3418  , df_denom=464, df_num=5

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0252  , p=0.8740  , df_denom=476, df_num=1
ssr based chi2 test:   chi2=0.0253  , p=0.8735  , df=1
likelihood ratio test: chi2=0.0253  , p=0.8735  , df=1
parameter F test:         F=0.0252  , p=0.8740  , df_denom=476, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.0360  , p=0.9646  , df_denom=473, df_num=2
ssr based chi2 test:   chi2=0.0728  , p=0.9642  , df=2
likelihood ratio test: chi2=0.0728  , p=0.9642  , df=2
parameter F test:         F=0.0360  , p=0.9646  , df_denom=473, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=0.0623  , p=0.9796  , df_denom=470, df_num=3
ssr based chi2 test:   chi2=0.1897  , p=0.9792  , df=3
likelihood ratio test: chi2=0.1897  , p=0.9792  , df=3
parameter F test:         F=0.0623  , p=0.9796  , df_denom=470, df_num=3

Granger Causality
nu

In [39]:
import pandas as pd
from statsmodels.tsa.stattools import grangercausalitytests

def perform_granger_full_output_and_tabulate(data):
    """
    Perform Granger causality tests for specified variable pairs, print full results,
    and tabulate the key statistics in a DataFrame, rounding results to three decimal places.
    """
    data['gdp_diff'] = data['gdp'].diff()  # Calculate the first difference of 'gdp'

    pairs_to_test = [('gdp_diff', 'epu'), ('gdp_diff', 'liquidity_1'), ('gdp_diff', 'liquidity_2'),
                     ('epu', 'liquidity_1'), ('epu', 'liquidity_2'), ('epu', 'gdp_diff'),
                     ('liquidity_1', 'gdp_diff'), ('liquidity_2', 'gdp_diff'), ('liquidity_1', 'epu'), ('liquidity_2', 'epu')]

    # Prepare an empty list to store results
    results = []

    for x, y in pairs_to_test:
        test_result = grangercausalitytests(data[[x, y]].dropna(), maxlag=5, verbose=False)
        
        # Extract required statistics for each lag and round them to three decimal places
        for lag, stats in test_result.items():
            chi2value = round(stats[0]['ssr_chi2test'][0], 3)
            p_value_chi2 = round(stats[0]['ssr_chi2test'][1], 3)
            f_value = round(stats[0]['params_ftest'][0], 3)
            p_value_f = round(stats[0]['params_ftest'][1], 3)
            results.append([f"{x} causing {y}", lag, chi2value, p_value_chi2, f_value, p_value_f])
    
    # Convert results into a DataFrame
    results_df = pd.DataFrame(results, columns=['Test', 'Lag', 'Chi2 Value', 'P-Value of Chi2', 'F Value', 'P-Value of F-test'])
    return results_df

# Example usage
# Assuming 'data' is your DataFrame with the necessary columns:
# results_df = perform_granger_full_output_and_tabulate(data)
# print(results_df)


In [40]:
perform_granger_full_output_and_tabulate(data)

,Test,Lag,Chi2 Value,P-Value of Chi2,F Value,P-Value of F-test
0,gdp_diff causing epu,1,0.138,0.710,0.137,0.712
1,gdp_diff causing epu,2,0.176,0.916,0.087,0.917
2,gdp_diff causing epu,3,0.217,0.975,0.071,0.975
3,gdp_diff causing epu,4,0.966,0.915,0.237,0.917
4,gdp_diff causing epu,5,1.264,0.939,0.247,0.941
5,gdp_diff causing liquidity_1,1,4.800,0.028,4.770,0.029
6,gdp_diff causing liquidity_1,2,3.855,0.146,1.907,0.150
7,gdp_diff causing liquidity_1,3,2.294,0.514,0.753,0.521
8,gdp_diff causing liquidity_1,4,9.778,0.044,2.398,0.049
9,gdp_diff causing liquidity_1,5,11.540,0.042,2.254,0.048


In [44]:
perform_granger_full_output_and_tabulate(data).to_excel('granger_result.xlsx')